# Import Libraries and Data

## This script contains the following:

**1. Import data and libraries**

**2. Data wrangling**

**3. Data cleaning**

**4. Plotting a choropleth**

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# Creating Path
path = r'C:\Users\anush\Zomato Banglore Restaurants'

In [3]:
# Importing .geojson file
bengaluru_geo = r'C:\Users\anush\Zomato Banglore Restaurants\02 Data\Original Data/BBMP.geojson'

In [4]:
# Importing zomato_cleaned.pkl File
df = pd.read_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'zomato_cleaned.pkl'))

In [5]:
df.head()

url  \
0  https://www.zomato.com/bangalore/jalsa-banasha...   
1  https://www.zomato.com/bangalore/spice-elephan...   
2  https://www.zomato.com/SanchurroBangalore?cont...   
3  https://www.zomato.com/bangalore/addhuri-udupi...   
4  https://www.zomato.com/bangalore/grand-village...   

                                             address                   name  \
0  942, 21st Main Road, 2nd Stage, Banashankari, ...                  Jalsa   
1  2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...         Spice Elephant   
2  1112, Next to KIMS Medical College, 17th Cross...        San Churro Cafe   
3  1st Floor, Annakuteera, 3rd Stage, Banashankar...  Addhuri Udupi Bhojana   
4  10, 3rd Floor, Lakshmi Associates, Gandhi Baza...          Grand Village   

  online_order book_table   rate  votes                             phone  \
0          Yes        Yes  4.1/5    775    080 42297555\r\n+91 9743772233   
1          Yes         No  4.1/5    787                      080 41714161   
2          Yes         No  3.8/5    918                    +91 9663487993   
3           No         No  3.7/5     88                    +91 9620009302   
4           No         No  3.8/5    166  +91 8026612447\r\n+91 9901210005   

       location            rest_type  \
0  Banashankari        Casual Dining   
1  Banashankari        Casual Dining   
2  Banashankari  Cafe, Casual Dining   
3  Banashankari          Quick Bites   
4  Basavanagudi        Casual Dining   

                                          dish_liked  \
0  Pasta, Lunch Buffet, Masala Papad, Paneer Laja...   
1  Momos, Lunch Buffet, Chocolate Nirvana, Thai G...   
2  Churros, Cannelloni, Minestrone Soup, Hot Choc...   
3                                        Masala Dosa   
4                                Panipuri, Gol Gappe   

                         cuisines approx_cost(for two people)  \
0  North Indian, Mughlai, Chinese                         800   
1     Chinese, North Indian, Thai                         800   
2          Cafe, Mexican, Italian                         800   
3      South Indian, North Indian                         300   
4        North Indian, Rajasthani                         600   

                                        reviews_list menu_item  \
0  [('Rated 4.0', 'RATED\n  A beautiful place to ...        []   
1  [('Rated 4.0', 'RATED\n  Had been here for din...        []   
2  [('Rated 3.0', "RATED\n  Ambience is not that ...        []   
3  [('Rated 4.0', "RATED\n  Great food and proper...        []   
4  [('Rated 4.0', 'RATED\n  Very good restaurant ...        []   

  listed_in(type) listed_in(city)  
0          Buffet    Banashankari  
1          Buffet    Banashankari  
2          Buffet    Banashankari  
3          Buffet    Banashankari  
4          Buffet    Banashankari

# Data Wrangling

## Explore the Data

In [6]:
# Inspect the data
print(df.info())
print(df.describe())

# Check for null values
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   url                          51717 non-null  object  
 1   address                      51717 non-null  object  
 2   name                         51717 non-null  object  
 3   online_order                 51717 non-null  category
 4   book_table                   51717 non-null  category
 5   rate                         51717 non-null  object  
 6   votes                        51717 non-null  int32   
 7   phone                        51717 non-null  object  
 8   location                     51717 non-null  category
 9   rest_type                    51717 non-null  category
 10  dish_liked                   51717 non-null  object  
 11  cuisines                     51717 non-null  object  
 12  approx_cost(for two people)  51717 non-null  object  
 13  r

In [7]:
# Replace non-numeric values ('Not Available') with NaN
df['approx_cost(for two people)'] = df['approx_cost(for two people)'].replace('Not Available', None)

# Remove commas and convert to float
df['approx_cost(for two people)'] = df['approx_cost(for two people)'].str.replace(',', '').astype(float)

# Convert the 'rate' column to numeric, handling non-numeric values
df['rate'] = df['rate'].str.split('/').str[0]
df['rate'] = pd.to_numeric(df['rate'], errors='coerce')

# Check for successful conversion
print(df[['approx_cost(for two people)', 'rate', 'votes']].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   approx_cost(for two people)  51371 non-null  float64
 1   rate                         41665 non-null  float64
 2   votes                        51717 non-null  int32  
dtypes: float64(2), int32(1)
memory usage: 1010.2 KB
None


## Remove Unnecessary Columns

In [8]:
df.drop(columns=['url', 'phone'], inplace=True)

## Categorize Columns

In [9]:
# Categorize the 'rate' into 'Low', 'Medium', and 'High'
df.loc[(df['rate'] == 0), 'rating_category'] = 'None'
df.loc[(df['rate'] > 0) & (df['rate'] <= 2), 'rating_category'] = 'Low'
df.loc[(df['rate'] > 2) & (df['rate'] <= 4), 'rating_category'] = 'Medium'
df.loc[(df['rate'] > 4), 'rating_category'] = 'High'

In [10]:
# Categorize 'approx_cost' into 'Low Price', 'Medium Price', and 'High Price'
df.loc[(df['approx_cost(for two people)'] == 0), 'price_category'] = 'None'
df.loc[(df['approx_cost(for two people)'] < 1000), 'price_category'] = 'Low Price'
df.loc[(df['approx_cost(for two people)'] >= 1000) & (df['approx_cost(for two people)'] < 3000), 'price_category'] = 'Medium Price'
df.loc[(df['approx_cost(for two people)'] >= 3000), 'price_category'] = 'High Price'

In [11]:
# Categorize 'votes' based on quartiles
df.loc[(df['votes'] == 0), 'vote_category'] = 'None'
df.loc[(df['votes'] > 0) & (df['votes'] <= 7), 'vote_category'] = 'Few Votes'
df.loc[(df['votes'] > 7) & (df['votes'] <= 41), 'vote_category'] = 'Moderately Voted'
df.loc[(df['votes'] > 41) & (df['votes'] <= 198), 'vote_category'] = 'Highly Voted'
df.loc[df['votes'] > 198, 'vote_category'] = 'Extremely Voted'

## Categorical Encoding

In [12]:
# Encode categorical variables
df['online_order'] = df['online_order'].map({'Yes': 1, 'No': 0})
df['book_table'] = df['book_table'].map({'Yes': 1, 'No': 0})

In [13]:
df.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)', 'rating_category',
       'price_category', 'vote_category'],
      dtype='object')

## Consistancy Checks

In [14]:
# Define a list of necessary columns for your analysis
columns = [
    'name',  # Name of the Restaurant
    'rate',  # Rating of the restaurant
    'votes',  # Number of votes
    'location',  # Location or neighbourhood of the restaurant
    'rest_type',  # Type of restaurant (e.g., Casual Dining, Quick Bites)
    'cuisines',  # Cuisine types offered by the restaurant
    'approx_cost(for two people)',  # Approximate cost for two people
    'listed_in(city)',  # City in which the restaurant is listed
    'rating_category',  # Rating category (e.g., Excellent, Good)
    'price_category',  # Price category (e.g., Low, Medium, High)
    'vote_category'  # Vote category based on number of votes
]

# Select only the necessary columns from the DataFrame
df_rec = df[columns]

In [15]:
df_rec

name  rate  votes  \
0                                                  Jalsa   4.1    775   
1                                         Spice Elephant   4.1    787   
2                                        San Churro Cafe   3.8    918   
3                                  Addhuri Udupi Bhojana   3.7     88   
4                                          Grand Village   3.8    166   
...                                                  ...   ...    ...   
51712  Best Brews - Four Points by Sheraton Bengaluru...   3.6     27   
51713                           Vinod Bar And Restaurant   NaN      0   
51714  Plunge - Sheraton Grand Bengaluru Whitefield H...   NaN      0   
51715  Chime - Sheraton Grand Bengaluru Whitefield Ho...   4.3    236   
51716                       The Nest - The Den Bengaluru   3.4     13   

                         location            rest_type  \
0                    Banashankari        Casual Dining   
1                    Banashankari        Casual Dining   
2                    Banashankari  Cafe, Casual Dining   
3                    Banashankari          Quick Bites   
4                    Basavanagudi        Casual Dining   
...                           ...                  ...   
51712                  Whitefield                  Bar   
51713                  Whitefield                  Bar   
51714                  Whitefield                  Bar   
51715  ITPL Main Road, Whitefield                  Bar   
51716  ITPL Main Road, Whitefield   Bar, Casual Dining   

                                     cuisines  approx_cost(for two people)  \
0              North Indian, Mughlai, Chinese                        800.0   
1                 Chinese, North Indian, Thai                        800.0   
2                      Cafe, Mexican, Italian                        800.0   
3                  South Indian, North Indian                        300.0   
4                    North Indian, Rajasthani                        600.0   
...                                       ...                          ...   
51712                             Continental                       1500.0   
51713                             Finger Food                        600.0   
51714                             Finger Food                       2000.0   
51715                             Finger Food                       2500.0   
51716  Finger Food, North Indian, Continental                       1500.0   

      listed_in(city) rating_category price_category     vote_category  
0        Banashankari            High      Low Price   Extremely Voted  
1        Banashankari            High      Low Price   Extremely Voted  
2        Banashankari          Medium      Low Price   Extremely Voted  
3        Banashankari          Medium      Low Price      Highly Voted  
4        Banashankari          Medium      Low Price      Highly Voted  
...               ...             ...            ...               ...  
51712      Whitefield          Medium   Medium Price  Moderately Voted  
51713      Whitefield             NaN      Low Price              None  
51714      Whitefield             NaN   Medium Price              None  
51715      Whitefield            High   Medium Price   Extremely Voted  
51716      Whitefield          Medium   Medium Price  Moderately Voted  

[51717 rows x 11 columns]

In [16]:
df_rec.isnull().sum()

name                               0
rate                           10052
votes                              0
location                           0
rest_type                          0
cuisines                           0
approx_cost(for two people)      346
listed_in(city)                    0
rating_category                10052
price_category                   346
vote_category                      0
dtype: int64

In [17]:
# Drop rows with missing values in the specified critical columns
df_cleaned = df_rec.dropna(subset=[
    'rate', 
    'approx_cost(for two people)', 
    'rating_category', 
    'price_category'
])

# Checking the shape after dropping rows with missing values
print(df_cleaned.shape)

(41418, 11)


In [18]:
df_cleaned.isnull().sum()

name                           0
rate                           0
votes                          0
location                       0
rest_type                      0
cuisines                       0
approx_cost(for two people)    0
listed_in(city)                0
rating_category                0
price_category                 0
vote_category                  0
dtype: int64

# Plot Choropleth Maps

## The average restaurant rating per ward

In [19]:
# Group by city and calculate the average rating
city_avg_rating = df_cleaned.groupby('listed_in(city)')['rate'].mean().reset_index()

# Rename columns for clarity
city_avg_rating.columns = ['City', 'Average Rating']

# Display the DataFrame
print(city_avg_rating)

                     City  Average Rating
0                     BTM        3.706618
1            Banashankari        3.666758
2       Bannerghatta Road        3.555875
3            Basavanagudi        3.707276
4               Bellandur        3.624249
5            Brigade Road        3.801146
6             Brookefield        3.562348
7           Church Street        3.802635
8         Electronic City        3.494068
9             Frazer Town        3.715514
10                    HSR        3.662022
11            Indiranagar        3.746762
12               JP Nagar        3.629362
13              Jayanagar        3.694956
14           Kalyan Nagar        3.648297
15           Kammanahalli        3.638986
16  Koramangala 4th Block        3.758974
17  Koramangala 5th Block        3.759513
18  Koramangala 6th Block        3.762359
19  Koramangala 7th Block        3.758035
20           Lavelle Road        3.775396
21                MG Road        3.800132
22           Malleshwaram        3

C:\Users\anush\AppData\Local\Temp\ipykernel_6256\3484356862.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  city_avg_rating = df_cleaned.groupby('listed_in(city)')['rate'].mean().reset_index()


In [20]:
# Define the mapping between City and ward_name
city_to_ward_mapping = {
    "BTM": "BTM Layout",
    "Banashankari": "Banashankari Temple ward",
    "Bannerghatta Road": "Arakere",
    "Basavanagudi": "Basavanagudi",
    "Bellandur": "Bellanduru",
    "Brigade Road": "Shantala Nagar",
    "Brookefield": "Dodda Nekkundi",
    "Church Street": "Gayithri Nagar",
    "Electronic City": "Singasandra",
    "Frazer Town": "Pulikeshinagar",
    "HSR": "HSR - Singasandra",
    "Indiranagar": "Hoysala Nagar",
    "JP Nagar": "J P Nagar",
    "Jayanagar": "Basavanagudi",
    "Kalyan Nagar": "Nagavara",
    "Kammanahalli": "Kammanahalli",
    "Koramangala 4th Block": "Koramangala",
    "Koramangala 5th Block": "Koramangala",
    "Koramangala 6th Block": "Koramangala",
    "Koramangala 7th Block": "Koramangala",
    "Lavelle Road": "Shanthi Nagar",
    "MG Road": "Shanthi Nagar",
    "Malleshwaram": "Devasandra",
    "Marathahalli": "Marathahalli",
    "New BEL Road": "Mattikere",
    "Old Airport Road": "HAL Airport",
    "Rajajinagar": "Rajaji Nagar",
    "Residency Road": "Yelahanka Satellite Town",
    "Sarjapur Road": "HSR - Singasandra",
    "Whitefield": "Kaveripura"
}

# Create a new column 'ward_name' based on the mapping
city_avg_rating['ward_name'] = city_avg_rating['City'].map(city_to_ward_mapping)

# Check the first few rows to confirm the new column
print(city_avg_rating.head())

                City  Average Rating                 ward_name
0                BTM        3.706618                BTM Layout
1       Banashankari        3.666758  Banashankari Temple ward
2  Bannerghatta Road        3.555875                   Arakere
3       Basavanagudi        3.707276              Basavanagudi
4          Bellandur        3.624249                Bellanduru


In [21]:
# Initialize the map, centred at an approximate location (latitude, longitude) with a high-level zoom
city_map = folium.Map(location=[12.9716, 77.5946], zoom_start = 10)  # Centered on Bangalore, India

In [22]:
import json

# Load the GeoJSON file properly into a Python dictionary
with open(r'C:\Users\anush\Zomato Banglore Restaurants\02 Data\Original Data/BBMP.geojson') as f:
    bengaluru_geo = json.load(f)

# Now you can check the keys of the GeoJSON properties
print(bengaluru_geo['features'][0]['properties'].keys())  # Check keys in GeoJSON

dict_keys(['KGISWardID', 'KGISWardCode', 'LGD_WardCode', 'KGISWardNo', 'KGISWardName', 'KGISTownCode'])


In [23]:
from branca.colormap import linear

# Create a color scale for the Average Rating
color_scale = linear.YlOrRd_09.scale(city_avg_rating['Average Rating'].min(), city_avg_rating['Average Rating'].max())
color_scale.caption = 'Average Restaurant Rating'

# Function to map ward names to colors
def get_color(ward_name):
    rating = city_avg_rating.loc[city_avg_rating['ward_name'] == ward_name, 'Average Rating'].values
    if len(rating) > 0:
        return color_scale(rating[0])  # Map rating to color scale
    else:
        return '#ffffff'  # White for missing data

# Initialize a map centered at Bengaluru
map_rate_ward = folium.Map(location=[12.9716, 77.5946], zoom_start=11)

# Add GeoJSON layer with interactivity
folium.GeoJson(
    data=bengaluru_geo,
    style_function=lambda feature: {
        'fillColor': get_color(feature['properties']['KGISWardName']),
        'color': 'black',  # Border color
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['KGISWardName'],  # Display Ward Name in tooltip
        aliases=['Ward:'],       # Alias for tooltip
        sticky=True              # Tooltip sticks to mouse
    )
).add_to(map_rate_ward)

# Add the color scale to the map
color_scale.add_to(map_rate_ward)

# Add a layer control to toggle layers
folium.LayerControl().add_to(map_rate_ward)

# Display the map
map_rate_ward

In [24]:
# Export .html File
map_rate_ward.save(os.path.join(path, '04 Analysis', 'Visualizations', '6.3.choropleth_rate_ward.html'), bbox_inches = 'tight')

### Insights from the Choropleth Map of Average Food Quality Ratings in Bengaluru Wards

The map visualizes the distribution of average food quality ratings across Bengaluru's wards:

- **High Ratings**: Wards like **Gayithri Nagar**, **Shantala Nagar**, **Shanthi Nagar**, **Yelahanka Satellite Town**, **Koramangala**, **Hoysala Nagar**, **HAL Airport**, **Devasandra**, **Pulikeshinagar**, **Basavanagudi** and **BTM Layout** have ratings above 3.70, indicating consistently high food quality.
- **Moderate Ratings**: **Banashankari Temple ward**, **HSR - Singasandra**, **Rajaji Nagar**, **Nagavara**, **Kammanahalli**, **Mattikere**, **J P Nagar**, **Kaveripura** and **Bellanduru** (3.60–3.70) offer reasonably good food quality.
- **Lower Ratings**: Wards like **Dodda Nekkundi**, **Arakere** and others (below 3.60) reflect lower food quality ratings.

#### Note:
To avoid data bias, rows with missing ratings were dropped, ensuring an accurate representation of food quality across the city.

## Choropleth Map of Total Votes in Bengaluru Wards

In [25]:
# Map `listed_in(city)` to `ward_name` and add a new column
df_cleaned['ward_name'] = df_cleaned['listed_in(city)'].map(city_to_ward_mapping)

# Display the updated DataFrame
print(df_cleaned)

                                                    name  rate  votes  \
0                                                  Jalsa   4.1    775   
1                                         Spice Elephant   4.1    787   
2                                        San Churro Cafe   3.8    918   
3                                  Addhuri Udupi Bhojana   3.7     88   
4                                          Grand Village   3.8    166   
...                                                  ...   ...    ...   
51709                         The Farm House Bar n Grill   3.7     34   
51711                                            Bhagini   2.5     81   
51712  Best Brews - Four Points by Sheraton Bengaluru...   3.6     27   
51715  Chime - Sheraton Grand Bengaluru Whitefield Ho...   4.3    236   
51716                       The Nest - The Den Bengaluru   3.4     13   

                         location            rest_type  \
0                    Banashankari        Casual Dining   
1      

C:\Users\anush\AppData\Local\Temp\ipykernel_6256\2430829375.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['ward_name'] = df_cleaned['listed_in(city)'].map(city_to_ward_mapping)


In [26]:
# Group the data by ward and calculate total votes
ward_votes = df_cleaned.groupby('ward_name')['votes'].sum().reset_index()
ward_votes.rename(columns={'votes': 'Total Votes'}, inplace=True)

# Create a colour scale for total votes
color_scale = linear.Blues_09.scale(ward_votes['Total Votes'].min(), ward_votes['Total Votes'].max())
color_scale.caption = 'Total Restaurant Votes'

# Function to map ward names to colours
def get_vote_color(ward_name):
    votes = ward_votes.loc[ward_votes['ward_name'] == ward_name, 'Total Votes'].values
    if len(votes) > 0:
        return color_scale(votes[0])  # Map total votes to color scale
    else:
        return '#808080'  # Grey for missing data

# Initialize a map centered at Bengaluru
map_votes_ward = folium.Map(location=[12.9716, 77.5946], zoom_start=11)

# Add GeoJSON layer with interactivity
folium.GeoJson(
    data=bengaluru_geo,
    style_function=lambda feature: {
        'fillColor': get_vote_color(feature['properties']['KGISWardName']),
        'color': 'black',  # Border color
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['KGISWardName'],  # Display Ward Name in tooltip
        aliases=['Ward:'],       # Alias for tooltip
        sticky=True              # Tooltip sticks to mouse
    )
).add_to(map_votes_ward)

# Add the color scale to the map
color_scale.add_to(map_votes_ward)

# Add a layer control to toggle layers
folium.LayerControl().add_to(map_votes_ward)

# Display the map
map_votes_ward

In [27]:
# Export .html File
map_votes_ward.save(os.path.join(path, '04 Analysis', 'Visualizations', '6.3.choropleth_votes_ward.html'), bbox_inches = 'tight')

### Insights from the Choropleth Map of Total Votes in Bengaluru Wards

The map displays the distribution of total votes across Bengaluru's wards:

- **High Vote Counts**: Wards like **Koramangala** show high vote counts, indicating areas with many customer reviews and engagement.
- **Moderate Vote Counts**: Wards such as **BTM Layout** and **Shanthi Nagar** have moderate vote counts, reflecting consistent customer participation.
- **Lower Vote Counts**: Areas like **Basavanagudi**, **Hoysala Nagar** and others have lower average votes, potentially indicating fewer customer engagement or reviews.

## Choropleth Map of Approximate Cost for Two People Across Wards

In [28]:
# Group the data by ward and calculate the average cost for two people
ward_cost = df_cleaned.groupby('ward_name')['approx_cost(for two people)'].mean().reset_index()
ward_cost.rename(columns={'approx_cost(for two people)': 'Avg Cost for Two'}, inplace=True)

# Create a purple color scale using `linear.Purples_09`
color_scale = linear.Purples_09.scale(ward_cost['Avg Cost for Two'].min(), ward_cost['Avg Cost for Two'].max())
color_scale.caption = 'Average Cost for Two People'

# Function to map ward names to colours based on average cost
def get_cost_color(ward_name):
    cost = ward_cost.loc[ward_cost['ward_name'] == ward_name, 'Avg Cost for Two'].values
    if len(cost) > 0:
        return color_scale(cost[0])  # Map average cost to color scale
    else:
        return '#A52A2A'  # Brown for missing data

# Initialize the map
map_cost_ward = folium.Map(location=[12.9716, 77.5946], zoom_start=11)

# Add GeoJSON layer with interactivity
folium.GeoJson(
    data=bengaluru_geo,
    style_function=lambda feature: {
        'fillColor': get_cost_color(feature['properties']['KGISWardName']),
        'color': 'black',  # Border colour
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['KGISWardName'],  # Display Ward Name in tooltip
        aliases=['Ward:'],       # Alias for tooltip
        sticky=True              # Tooltip sticks to mouse
    )
).add_to(map_cost_ward)

# Add the colour scale to the map
color_scale.add_to(map_cost_ward)

# Add a layer control to toggle layers
folium.LayerControl().add_to(map_cost_ward)

# Display the map
map_cost_ward

In [29]:
# Export .html File
map_cost_ward.save(os.path.join(path, '04 Analysis', 'Visualizations', '6.3.choropleth_cost_ward.html'), bbox_inches = 'tight')

### Insights from the Choropleth Map of Approximate Cost for Two People Across Wards

The map highlights the distribution of average dining costs across Bengaluru's wards:

- **High-Cost Areas**: Wards like **Gayithri Nagar**, **Shantala Nagar**, **Shanthi Nagar** and **Yelahanka Satellite Town** exhibit higher average costs, reflecting upscale dining options and premium restaurants.  
- **Moderate-Cost Areas**: Wards such as **Hoysala Nagar**, **HAL Airport**, **Kaveripura**, **Devasandra**, **Pulikeshinagar** and **Bellanduru** showcase moderate costs, balancing affordability with variety.  
- **Low-Cost Areas**: Wards like **Dodda Nekkundi**, **Singasandra**, **Koramangala**, **HSR - Singasandra** and others feature lower average costs, indicating budget-friendly dining options.

#### Note:  
Rows with missing cost data were excluded to ensure accuracy and avoid data bias. This analysis helps identify the affordability and pricing trends across Bengaluru's neighbourhoods.

# Discussion of Results

## Implications for Average Food Quality Ratings

* Customers looking for high-quality dining experiences may prefer high-rated wards.
* Restaurant owners in lower-rated wards may improve food quality to attract more customers.

## Implications for Total Votes

* High vote counts in areas like Koramangala suggest it is a hub of customer activity and restaurant competition.
* Lower vote counts in certain wards may indicate potential growth opportunities for new restaurants or the need for better marketing strategies.

## Implications for Approximate Cost for Two People

* High-cost wards may attract customers seeking luxurious dining experiences.
* Low-cost wards could see an influx of budget-conscious diners, creating opportunities for casual dining and quick-service restaurants.


## Addressing Research Questions

**1. What are the key areas for high food quality in Bengaluru?**
* Yes, the analysis identifies wards like Koramangala and Shantala Nagar as hubs of high-quality dining options.

**2. Where do customers actively engage with restaurants?**
* Yes, the map highlights wards like Koramangala and BTM Layout as areas of high customer engagement.

**3. How do dining costs vary across Bengaluru?**
* Yes, the analysis categorizes wards into high-cost, moderate-cost, and low-cost areas, providing insights into affordability.

## Does the analysis lead to new research questions?

**1. What factors drive customer engagement in certain wards (e.g., Koramangala)?**

**2. Are there specific cuisines or dining experiences contributing to higher ratings in high-quality wards?**

**3. Why are some wards with lower costs still underperforming in terms of engagement and votes?**

**4. Do socio-economic factors of wards influence dining costs and customer preferences?**

This analysis provides actionable insights into the restaurant landscape in Bengaluru, helping stakeholders like customers, restaurants, and city planners make informed decisions. It also opens doors to further research to explore the underlying factors influencing these patterns and their implications for the city's food culture.

In [30]:
df_cleaned.head()

name  rate  votes      location            rest_type  \
0                  Jalsa   4.1    775  Banashankari        Casual Dining   
1         Spice Elephant   4.1    787  Banashankari        Casual Dining   
2        San Churro Cafe   3.8    918  Banashankari  Cafe, Casual Dining   
3  Addhuri Udupi Bhojana   3.7     88  Banashankari          Quick Bites   
4          Grand Village   3.8    166  Basavanagudi        Casual Dining   

                         cuisines  approx_cost(for two people)  \
0  North Indian, Mughlai, Chinese                        800.0   
1     Chinese, North Indian, Thai                        800.0   
2          Cafe, Mexican, Italian                        800.0   
3      South Indian, North Indian                        300.0   
4        North Indian, Rajasthani                        600.0   

  listed_in(city) rating_category price_category    vote_category  \
0    Banashankari            High      Low Price  Extremely Voted   
1    Banashankari            High      Low Price  Extremely Voted   
2    Banashankari          Medium      Low Price  Extremely Voted   
3    Banashankari          Medium      Low Price     Highly Voted   
4    Banashankari          Medium      Low Price     Highly Voted   

                  ward_name  
0  Banashankari Temple ward  
1  Banashankari Temple ward  
2  Banashankari Temple ward  
3  Banashankari Temple ward  
4  Banashankari Temple ward

In [31]:
# Exporting the Dataframe
df_cleaned.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'Zomato_with_ward_name.pkl'))

In [32]:
# Exporting Dataframe
df_cleaned.to_csv(os.path.join(path, '02 Data','Prepared Data', 'Zomato_with_ward_name.csv'))